In [17]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import math
import time

In [18]:
#SCRAPPING
#extract data from tags
def extractData(soup, tag, name):
    elements = soup.find_all(tag,  class_=name )
    return [i.text.strip() for i in elements ]

In [19]:
#remove symbols from price
def cleanPrice(raw):
    return raw.replace('₹', '').replace(',', '')

In [20]:
company = []
processer = []
ram = []
os = []
ssd = []
hdd = []
ssize = []
price = []
ignore=[]

#scrapping data from accessible pages
for i in range(1,41):
    data = requests.get(f'https://www.flipkart.com/search?q=laptops&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&as-pos=1&as-type=HISTORY&page={i}')
    time.sleep(5)
    soup = BeautifulSoup(data.content,'html')
    
    com = extractData(soup, 'div', '_4rR01T')
    company.extend([i.split()[0] for i in com])

    for specs in extractData(soup, 'li', 'rgWa7D'):
        detail = specs.split()
        if 'RAM' and 'SSD' and 'Storage' in detail:
            ignore.append(detail)
        elif 'Display' and 'nits' in detail:
            ignore.append(detail)
        elif detail.count('Processor') == 1:
            os.append(detail[0])
            processer.append(detail[1]+' '+detail[2])
        elif detail.count('RAM') == 1 and detail.count('GB'):
            ram.append(detail[0])
        elif detail.count('Display') == 1 and detail.count('cm') == 1:
            ssize.append(detail[0])
        elif detail.count('HDD') == 0 and detail.count('SSD')==1:
            ssd.append(detail[detail.index('SSD')-2]+detail[detail.index('SSD')-1])
            hdd.append(0)
        elif detail.count('HDD') == 1 and detail.count('SDD')==0:
            hdd.append(detail[detail.index('HDD')-2]+detail[detail.index('HDD')-1])
            ssd.append(0)
        elif detail.count('HDD') == 1 and detail.count('SSD')==1:
            hdd.append(detail[detail.index('HDD')-2]+detail[detail.index('HDD')-1])
            ssd.append(detail[detail.index('SSD')-2]+detail[detail.index('SSD')-1])
                            
    pr = extractData(soup, 'div','_30jeq3 _1_WHN1')
    price.extend([cleanPrice(p) for p in pr])    

In [21]:
print(len(company))
print(len(processer))
print(len(ram))
print(len(os))
print(len(hdd))
print(len(ssd))
print(len(ssize))
print(len(price))

504
506
504
506
467
467
504
504


In [22]:
#changing the data to put in dataframe
CurrentP = ['Core i3','Core i5','Core i7','Core i9','M1 Max','M1 Pro','M1 Processor','Ryzen 3','Ryzen 5','Ryzen 7','Ryzen 9']
ProcesserNew = []
for i in processer:
    if i in CurrentP:
        ProcesserNew.append(i)
    else:
        ProcesserNew.append('Others')
        
RamNew = []
for i in ram:
    if (i.isdigit()):
        RamNew.append(int(i))

OsNew = []
standard = ['Intel','AMD','Apple','MediaTek']
for i in os:
    if i in standard:
        OsNew.append(i)
    else:
        OsNew.append('Others')

HddNew = []
for i in hdd:
    if i=='1TB':
        HddNew.append(int(1024))
    elif i=='500GB':
        HddNew.append(int(516))
    else:
        HddNew.append(0)

SsdNew = []
for i in ssd:
    i = str(i)
    if '128GB' in i:
        SsdNew.append(int(128))
    elif '256GB' in i:
        SsdNew.append(int(256))
    elif '512GB' in i:
        SsdNew.append(int(512))
    elif '1TB' in i:
        SsdNew.append(int(1024))
    elif '2TB' in i:
        SsdNew.append(int(2000))
    elif '4TB' in i:
        SsdNew.append(int(4000))
    else:
        SsdNew.append(0)
    
        
FinalPrice = []
for i in price:
    FinalPrice.append(int(i))
    
print(len(company))
print(len(ProcesserNew))
print(len(RamNew))
print(len(OsNew))
print(len(HddNew))
print(len(SsdNew))
print(len(ssize))
print(len(FinalPrice))


504
506
504
506
467
467
504
504


In [25]:
#adjusting the size 
size = 460
company = company[:size]
ProcesserNew = ProcesserNew[:size]
RamNew = RamNew[:size]
OsNew = OsNew[:size]
HddNew = HddNew[:size]
SsdNew = SsdNew[:size]
SsizeNew = ssize[:size]
FinalPrice = FinalPrice[:size]

In [26]:
#converting to a dataframe
data = {
    'Company':company,
    'Processor':ProcesserNew,
    'RAM' : RamNew,
    'OperatingSystem':OsNew,
    'HDD':HddNew,
    'SSD': SsdNew,
    'ScreenSize':SsizeNew,
    'Price':FinalPrice
}
df = pd.DataFrame(data)
df

,Company,Processor,RAM,OperatingSystem,HDD,SSD,ScreenSize,Price
0,Primebook,Others,4,MediaTek,0,512,29.46,12990
1,Lenovo,Core i5,8,Intel,0,512,39.62,49990
2,HP,Others,4,Intel,0,512,39.62,27490
3,MSI,Core i5,8,Intel,0,512,39.62,46990
4,Primebook,Others,4,MediaTek,0,512,29.46,16990
...,...,...,...,...,...,...,...,...
455,AXL,Ryzen 7,4,AMD,0,512,39.62,17990
456,Acer,Core i5,16,Intel,0,512,40.89,97290
457,MSI,Others,64,Intel,0,512,43.94,420000
458,Acer,Core i7,16,Intel,0,512,39.62,54990


In [15]:
df_new = pd.DataFrame(df)
df_new.to_csv('laptops_data.csv', index=False)